<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>

# 内存高效的模型权重加载

- 本笔记本提供了在 GPU（或 CPU）内存有限的情况下加载较大的预训练或微调模型的技巧
- 具体来说，它侧重于使用 `torch.save(model.state_dict(), "model.pth")` 保存模型的情况（例如，在第 5-7 章中），并希望稍后在新会话中加载它以继续进行预训练或额外微调
- 虽然示例使用了 LLM，但本笔记本中解释的方法是通用的，适用于加载任何 PyTorch 模型，而不仅仅是 LLM

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/memory-efficient-loading/memory-efficient-loading.webp" width="800px">

In [1]:
from importlib.metadata import version

pkgs = [
    "torch",
]
for p in pkgs:
    print(f"{p} version: {version(p)}")

memory_profiler version: 0.61.0
torch version: 2.4.1+cu121


&nbsp;
## 1. 基准测试工具

- 首先，让我们定义一些实用程序代码来跟踪 VRAM（GPU 内存）
- 稍后，我们还将介绍一个跟踪主系统 RAM（CPU 内存）的工具
- 稍后应用这些函数时，它们的用途就会变得清晰

In [2]:
import gc
import time
import torch


def start_memory_tracking():
    """Initialize GPU memory tracking."""
    if torch.cuda.is_available():
        torch.cuda.reset_peak_memory_stats()
    else:
        print("This notebook is intended for CUDA GPUs but CUDA is not available.")

def print_memory_usage():
    max_gpu_memory = torch.cuda.max_memory_allocated() / (1024 ** 3)  # Convert bytes to GB
    print(f"Maximum GPU memory allocated: {max_gpu_memory:.1f} GB")

def cleanup():
    gc.collect()
    torch.cuda.empty_cache()
    time.sleep(3)  # some buffer time to allow memory to clear
    torch.cuda.reset_peak_memory_stats()
    max_memory_allocated = torch.cuda.max_memory_allocated(device) / (1024 ** 3)
    print(f"Maximum GPU memory allocated: {max_memory_allocated:.1f} GB")

&nbsp;
## 2. 模型设置

- 此代码部分设置模型本身
- 在这里，我们使用 “大型” GPT-2 模型来使事情变得更有趣（您可以使用“gpt2-small (124M)”来降低此笔记本的内存要求和执行时间）

In [3]:
from previous_chapters import GPTModel


BASE_CONFIG = {
    "vocab_size": 50257,     # Vocabulary size
    "context_length": 1024,  # Context length
    "drop_rate": 0.0,        # Dropout rate
    "qkv_bias": True         # Query-key-value bias
}

model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

CHOOSE_MODEL = "gpt2-xl (1558M)"

BASE_CONFIG.update(model_configs[CHOOSE_MODEL])

- 现在，让我们看看 GPU 内存功能的实际作用：

In [4]:
start_memory_tracking()


model = GPTModel(BASE_CONFIG)
device = torch.device("cuda")
model.to(device)

print_memory_usage()

Maximum GPU memory allocated: 6.4 GB


- 另外，我们通过传入一些示例张量来确保模型运行正常

In [5]:
# Test if the model works (no need to track memory here)
test_input = torch.tensor([[1, 2, 3]]).to(device)
model.eval()

with torch.no_grad():
    model(test_input)

- 接下来，假设我们正在预训练模型并将其保存以供日后使用
- 为简单起见，我们在这里跳过实际的预训练，只保存初始化的模型（但同样的概念适用）

In [6]:
# Training code would go here...

model.train()
torch.save(model.state_dict(), "model.pth")

- 最后，我们在 Python 会话中删除模型和示例张量以重置 GPU 内存

In [7]:
del model, test_input
cleanup()

Maximum GPU memory allocated: 0.0 GB


&nbsp;
## 3. 权重加载

- 现在开始有趣的部分，我们加载预训练的模型权重
- 让我们看看加载之前保存的模型需要多少 GPU 内存

In [8]:
# Then load pretrained weights

start_memory_tracking()

model = GPTModel(BASE_CONFIG)
model.to(device)

model.load_state_dict(
    torch.load("model.pth", map_location=device, weights_only=True)
)
model.to(device)
model.eval();

print_memory_usage()

Maximum GPU memory allocated: 12.8 GB


- 请注意，内存是上一个会话的 2 倍
- 这是因为我们在内存中两次使用了相同的模型，但只持续了很短的时间：
- 第一次通过 `model.to(device)`
- 第二次通过代码行 `model.load_state_dict(torch.load("model.pth", map_location=device, weights_only=True))`；最终，加载的模型权重将被复制到模型中，并且 `state_dict` 将被丢弃，但在很短的时间内，我们在内存中同时拥有主模型和加载的 `state_dict`
- 其余部分将重点介绍如何解决此问题
- 但首先，让我们测试模型并重置 GPU 内存


In [9]:
# Test if the model works (no need to track memory here)
test_input = torch.tensor([[1, 2, 3]]).to(device)
model.eval()

with torch.no_grad():
    model(test_input)

del model, test_input
cleanup()

Maximum GPU memory allocated: 0.0 GB


&nbsp;
## 4. 按顺序加载权重

- 如上一节所强调的，解决模型权重在 GPU 内存中出现两次的问题的一种方法是按顺序加载模型
- 下面，我们：
  - 首先将模型加载到 GPU 内存中
  - 然后将模型权重加载到 CPU 内存中
  - 最后将每个参数逐一复制到 GPU 内存中

In [10]:
start_memory_tracking()

model = GPTModel(BASE_CONFIG).to(device)

state_dict = torch.load("model.pth", map_location="cpu", weights_only=True)

print_memory_usage()

# Sequentially copy weights to the model's parameters
with torch.no_grad():
    for name, param in model.named_parameters():
        if name in state_dict:
            param.copy_(state_dict[name].to(device))
        else:
            print(f"Warning: {name} not found in state_dict.")

print_memory_usage()

Maximum GPU memory allocated: 6.4 GB
Maximum GPU memory allocated: 6.7 GB


- 如上所示，内存使用量比以前低得多
- 请注意，内存从 6.4 GB 增加到 6.7 GB，因为最初，内存中只有模型，然后内存中有了模型加上 1 个参数张量（我们暂时将参数张量移动到 GPU，以便我们可以使用 `".to"` 将其分配给模型）
- 总体而言，这是一个显着的改进
- 再次，让我们简要测试模型，然后重置 GPU 内存以进行下一部分

In [11]:
# Test if the model works (no need to track memory here)
test_input = torch.tensor([[1, 2, 3]]).to(device)
model.eval()

with torch.no_grad():
    model(test_input)

del model, test_input, state_dict, param
cleanup()

Maximum GPU memory allocated: 0.0 GB


&nbsp;
## 5. 使用低 CPU 内存加载模型

- 在上一节中，我们通过先将权重（`state_dict`）加载到 CPU 内存中，然后再将它们逐个复制到模型中，从而减少了 GPU 内存的使用
- 但是，如果我们的 CPU 内存有限，该怎么办？
- 本节使用 PyTorch 所谓的 `"meta"` 设备方法在具有大 GPU 内存但小 CPU 内存的机器上加载模型
- 但首先，让我们定义一个便利函数来监控 CPU 内存

In [12]:
import os
import psutil
from threading import Thread


def memory_usage_in_gb(func, *args, **kwargs):
    process = psutil.Process(os.getpid())

    # Measure the baseline memory usage before running the function
    baseline_mem = process.memory_info().rss / 1024 ** 3  # in GB

    # Start monitoring memory in a separate thread
    mem_usage = []
    done = False

    def monitor_memory():
        while not done:
            mem_usage.append(process.memory_info().rss / 1024 ** 3)  # Convert to GB
            time.sleep(0.1)

    t = Thread(target=monitor_memory)
    t.start()

    # Run the function
    func(*args, **kwargs)

    # Stop monitoring
    done = True
    t.join()

    peak_mem_usage_gb = max(mem_usage) - baseline_mem
    return peak_mem_usage_gb


- 首先，让我们跟踪上一节中顺序权重加载方法的 CPU 内存

In [13]:
def load_sequentially():
    start_memory_tracking()

    model = GPTModel(BASE_CONFIG).to(device)

    state_dict = torch.load("model.pth", map_location="cpu", weights_only=True)

    print_memory_usage()

    # Sequentially copy weights to the model's parameters
    with torch.no_grad():
        for name, param in model.named_parameters():
            if name in state_dict:
                param.copy_(state_dict[name].to(device))
            else:
                print(f"Warning: {name} not found in state_dict.")

    print_memory_usage()


peak_memory_used = memory_usage_in_gb(load_sequentially)
print(f"-> Maximum CPU memory allocated: {peak_memory_used:.1f} GB")

Maximum GPU memory allocated: 6.4 GB
Maximum GPU memory allocated: 6.7 GB
-> Maximum CPU memory allocated: 6.3 GB


- 现在，假设我们有一台 CPU 内存较低但 GPU 内存较大的机器
- 我们可以通过引入 PyTorch 所谓的 "meta" 设备来权衡 CPU 内存和 GPU 内存的使用情况
- PyTorch 的元设备是一种特殊的设备类型，它允许您创建张量而无需为其数据分配实际内存，从而有效地创建 "meta" 张量
- 这对于模型分析或架构定义等任务很有用，在这些任务中您需要张量形状和类型而不需要内存分配的开销

In [14]:
def load_sequentially_with_meta():
    start_memory_tracking()

    with torch.device("meta"):
        model = GPTModel(BASE_CONFIG)

    model = model.to_empty(device=device)

    state_dict = torch.load("model.pth", map_location=device, weights_only=True)

    print_memory_usage()

    # Sequentially copy weights to the model's parameters
    with torch.no_grad():
        for name, param in model.named_parameters():
            if name in state_dict:
                param.copy_(state_dict[name])
            else:
                print(f"Warning: {name} not found in state_dict.")

    print_memory_usage()

peak_memory_used = memory_usage_in_gb(load_sequentially_with_meta)
print(f"-> Maximum CPU memory allocated: {peak_memory_used:.1f} GB")

Maximum GPU memory allocated: 12.8 GB
Maximum GPU memory allocated: 12.8 GB
-> Maximum CPU memory allocated: 1.3 GB


- 如上所示，通过在元设备上创建模型并将权重直接加载到 GPU 内存中，我们有效地降低了 CPU 内存需求
- 有人可能会问：“那么顺序权重加载是否仍然必要？与原始方法相比如何？”
- 让我们检查一下简单的 PyTorch 权重加载方法进行比较（来自本笔记本中的第一个权重加载部分）：

In [15]:
def baseline():
    start_memory_tracking()

    model = GPTModel(BASE_CONFIG)
    model.to(device)

    model.load_state_dict(torch.load("model.pth", map_location=device, weights_only=True))
    model.to(device)
    model.eval();

    print_memory_usage()

peak_memory_used = memory_usage_in_gb(baseline)
print(f"-> Maximum CPU memory allocated: {peak_memory_used:.1f} GB")

Maximum GPU memory allocated: 12.8 GB
-> Maximum CPU memory allocated: 4.4 GB


- 如上所示，不使用元设备的“简单”权重加载会占用更多内存
- 换句话说，如果你的机器 CPU 内存有限，你可以使用元设备方法直接将模型权重加载到 GPU 内存中，以减少峰值 CPU 内存使用率

&nbsp;
## 6. 使用 `mmap=True`（推荐）

- 作为中级或高级 `torch.load` 用户，您可能想知道这些方法与 PyTorch 中的 `mmap=True` 设置相比如何
- PyTorch 中的 `mmap=True` 设置启用内存映射文件 I/O，这允许张量直接从磁盘存储访问数据，从而在 RAM 有限的情况下不将整个文件加载到 RAM 中，从而减少内存使用量
- 另请参阅 [mikaylagawarecki](https://github.com/rasbt/LLMs-from-scratch/issues/402) 的有用评论
- 乍一看，它可能看起来不如上面的顺序方法高效：

In [37]:
def best_practices():
  with torch.device("meta"):
      model = GPTModel(BASE_CONFIG)

  model.load_state_dict(
      torch.load("model.pth", map_location=device, weights_only=True, mmap=True),
      assign=True
  )

  print_memory_usage()

peak_memory_used = memory_usage_in_gb(best_practices)
print(f"-> Maximum CPU memory allocated: {peak_memory_used:.1f} GB")

Maximum GPU memory allocated: 6.4 GB
-> Maximum CPU memory allocated: 5.9 GB


- CPU RAM 使用率如此之高的原因是这台机器上有足够的 CPU RAM
- 但是，如果你在 CPU RAM 有限的机器上运行此程序，则 `mmap` 方法将使用较少的内存

&nbsp;
## 7. 其他方法

- 本笔记本重点介绍在 PyTorch 中加载权重的简单内置方法
- 对于有限 CPU 内存情况，建议的方法是 `mmap=True` 方法，解释得足够清楚
- 或者，另一种选择是使用蛮力方法，分别保存和加载每个权重张量：

In [13]:
model = GPTModel(BASE_CONFIG)
# Assume `model` is your trained model
state_dict = model.state_dict()

# Create a directory to store individual parameter files
os.makedirs("model_parameters", exist_ok=True)

# Save each parameter tensor separately
for name, param in state_dict.items():
    torch.save(param.cpu(), f"model_parameters/{name}.pt")

del model

In [16]:
def load_individual_weights():

    start_memory_tracking()

    with torch.device("meta"):
        model = GPTModel(BASE_CONFIG)

    model = model.to_empty(device=device)

    print_memory_usage()
    param_dir = "model_parameters"

    with torch.no_grad():
        for name, param in model.named_parameters():
            weight_path = os.path.join(param_dir, f"{name}.pt")
            if os.path.exists(weight_path):
                param_data = torch.load(weight_path, map_location="cpu", weights_only=True)
                param.copy_(param_data)
                del param_data  # Free memory
            else:
                print(f"Warning: {name} not found in {param_dir}.")

    print_memory_usage()


peak_memory_used = memory_usage_in_gb(load_individual_weights)
print(f"-> Maximum CPU memory allocated: {peak_memory_used:.1f} GB")

Maximum GPU memory allocated: 6.4 GB
Maximum GPU memory allocated: 6.4 GB
-> Maximum CPU memory allocated: 0.3 GB
